In [ ]:
import os
import glob
import requests
from tqdm import tqdm
import numpy as np

CHUNK_SIZE = 1024
headers = {
    "Accept": "audio/mpeg",
    "Content-Type": "application/json",
    # "xi-api-key": "sk_7205b11e0ebbab711f1a8ea74f1801e601b536ffd7049a2b", # old
    "xi-api-key": "sk_3ec0c10190d82a4fc3b11bf6c1e7fbe872877899c7fad2ca", # new
}

def process_audio(text, audiofile, speaker_id):
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{speaker_id}"
    data = {
        "text": text,
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.5
        }
    }
    response = requests.post(url, json=data, headers=headers)
    with open(audiofile, 'wb') as f:
        for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
            if chunk:
                f.write(chunk)
                
textpath = "../../Dataset/CMU-ARCTIC/transciptions.txt"
file1 = open(textpath, 'r')
Lines = file1.readlines()
transcriptions = {line.split(" ")[1]: line.split('"')[1] for line in Lines}

In [ ]:
base_dir = "./PD-AST/"
voiceids = {
    "Adam": "pNInz6obpgDQGcFmaJgB",
    
    # "TNI": "MpmvzVG7V9x1NbkUPjVk",
    # "RRBI": "Q1Pm5cykKaBq2v2HckOj",
    # "HKK": "SX85hGCDoEZvc5fEmxgv",
    # "SVBI": "YF1rYo6M2Yl40ZQnhmUX",
    
    # old account
    # "ASI": "Dnq2rswSKxxLdUYE9ARt",
    # "BDL": "dw501enouzsh9uIAnbAs",
    
    # new account
    # "ASI": "mfXp2UGhYy9CVfhbR80T",
    # "BDL": "22EbXCP4IsFd37zg8M2t",
    # "SLT": "iuutCMpgpPl7aa4Zerxj",
    
    # full prompt with 44.1kHz
    "ASI": "n7UG1mrDAGhkM6Jp4ad3",
    "TNI": "7MSSIGwrn7niZHnmFZQ1",
    "HKK": "9I8SKP1WTmBG3o8WyMt5",
    "SLT": "Sg4hcrJYcxtUZzg3hQlC",
    "BDL": "Z7R3b2X0NK1pfSp01AtA",
    
    # only valid
    # "ASI": "NsrvuJ6F9bOsRfwed1ET",
    # "SLT": "Zoy7f4m2oJCfP4Sph2F0",
    # "HKK": "zxqygYvyeaBnXQjljDd4",
    
    # valid + shuffle
    # "ASI": "via1ElYhAo4KUwwgdsss", # 0.2
    # "ASI": "3nywFZs7cpC9maGEoA8Z", # 0.5
    # "ASI": "Uo7THUW6T8evBkKwCQ2Y", # 1.0
    
    # wer
    # "ASI": "ynnlZ86VAOmtW6jYxGFx",
}

files = glob.glob("./LLM_responses/08-A_multi-lingual_text/*")
# files = glob.glob("./LLM_responses/08-A_multi-lingual_text GPT3-Turbo/*")
files.sort()

In [ ]:
files1 = glob.glob("./LLM_responses/08-A_multi-lingual_text/*")
files1.sort()
files2 = glob.glob("./LLM_responses/08-A_multi-lingual_text GPT3-Turbo/*")
files2.sort()

In [ ]:
for i in range(len(files1)):
    path1 = files1[i]
    path2 = files2[i]
    if not('arctic_a0024' in path1):
        continue
    try:
        data = np.load(path1, allow_pickle=True).item()
        text1 = data["Korean"]
        data = np.load(path2, allow_pickle=True).item()
        text2 = data["Korean"]
        print(text1)
        print(text2)
    except:
        pass

In [ ]:
# speakers = ["HKK"]
speakers = ["SLT"]
# speakers = ["ASI"]
# speakers = ["TNI", "HKK"]
# accents = ["Hindi"]
# accents = ["English", "Hindi", "Japanese", "Korean"]
# accents = ["Korean"]
accents = ["Korean"]

In [ ]:
save = False
for spk in speakers:
    spkid = voiceids[spk]
    spk_dir = base_dir + f"{spk}/"
    os.makedirs(spk_dir, exist_ok=True)
    # for acc in accents:
    for acc in accents:
        acc_dir = spk_dir + f"{acc}/"
        os.makedirs(acc_dir, exist_ok=True)
        for path in tqdm(files):
            savefile = acc_dir + os.path.basename(path)[:-4] + ".wav"
            if os.path.exists(savefile):
                continue
            data = np.load(path, allow_pickle=True).item()
            text = data[acc]
            if save:
                process_audio(text, savefile, spkid)

---
---
---

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import librosa
import torchaudio
import IPython
import shutil
def play_audio(data, rate):
    IPython.display.display(IPython.display.Audio(data=data,rate=rate))
    
import sys
sys.path.append("../cuhksz-phd/sho_util/pyfiles/")
from get_mel_spectrogram import audio2mel
from basic import plot_spectrogram

fs = 44100

In [ ]:
# spk = "ASI"
# spk = "HKK"
spk = "SLT"
spk_dir = base_dir + f"{spk}/"
accents = ["English", "Hindi"]

In [ ]:
baselines = [os.path.basename(a) for a in glob.glob(spk_dir + "Hindi/*")]
baselines.sort()

In [ ]:
save_spk_dir = f"./audiosamples/elevenlabs/dataset/{spk}/"
copy = False
os.makedirs(save_spk_dir, exist_ok=True)
# for bl in baselines:
for bl in np.random.choice(baselines, 10, False):
    path = f"./LLM_responses/08-A_multi-lingual_text/{bl[:-4]}.npy"
    data = np.load(path, allow_pickle=True).item()
    print("English: ",data["English"])
    # for acc in np.array(accents)[np.random.choice(np.arange(2), 2, False)]:
    # for acc in accents:
    for acc in accents[1:]:
        path = f"{spk_dir}{acc}/{bl}"
        if copy:
            tgt = save_spk_dir + f"{acc}_{bl}"
            shutil.copyfile(path, tgt)
        x, sr = torchaudio.load(path)
        x = np.array(x)[0]
        print(acc, ": ", data[acc])
        # print(acc, sr)
        play_audio(x, sr)
    if spk in ["SLT"]:
        path = f"/Users/shoahoshoaho/Git/Dataset/CMU-ARCTIC/cmu_us_{spk.lower()}_arctic/wav/{bl}"
    else:
        path = f"/Users/shoahoshoaho/Git/Dataset/L2-ARCTIC_v5/{spk}/wav/{bl}"
    x, sr = torchaudio.load(path)
    if copy:
        tgt = save_spk_dir + f"groundtruth_{bl}"
        shutil.copyfile(path, tgt)
    x = np.array(x)[0]
    # print("ground truth", sr)
    # play_audio(x, sr)
    print("-----------------------------")

In [ ]:
import torch
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(10, 5))
# path = "./PD-AST/ASI/English/arctic_a0002.wav"
# path = "/Users/shoahoshoaho/Git/Dataset/L2-ARCTIC_v5/ASI/wav/arctic_a0002.wav"
path = "/Users/shoahoshoaho/Git/Dataset/CMU-ARCTIC/cmu_us_slt_arctic/wav/arctic_a0001.wav"
x, _ = torchaudio.load(path)
x = np.array(x)[0]
# x, _ = librosa.load("./PD-AST/ASI/Hindi/arctic_a0002.wav", fs)
play_audio(x, _)
stft = np.log(np.abs(librosa.stft(x, )))
plot_spectrogram(stft, fig, (1, 2, 1), title="new")

# path = "./PD-AST_old/ASI/English/arctic_a0002.wav"
# path = "/Users/shoahoshoaho/Git/Dataset/L2-ARCTIC_v5/ASI/wav/arctic_a0002.wav"
# path = "./audiosamples/l2-ARCTIC/ASI_0.wav"
# path = "/Users/shoahoshoaho/Git/Dataset/CMU-ARCTIC/cmu_us_slt_arctic/wav/arctic_a0001.wav"
path = "./PD-AST/SLT/English/arctic_a0001.wav"
x, _ = torchaudio.load(path)
resampler = torchaudio.transforms.Resample(_, 16000, dtype=x.dtype)
x = resampler(x)
x = np.array(x)[0]
# x, _ = librosa.load("./PD-AST_old/ASI/Hindi/arctic_a0002.wav", fs)
play_audio(x, 16000)
stft = np.log(np.abs(librosa.stft(x)))
plot_spectrogram(stft, fig, (1, 2, 2), title="old")

In [ ]:
fs